## Классификация комментариев с помощью LSTM

1. Для улучшения качества работы модели можно поиграться с параметрами num_layers, hidden_size и embedding_dim: например, попробовать увеличить одновременно все 3 параметра и т.д.


2. чтобы подавать на вход модели только топ-1000 часто-встречающихся слов, надо изменить строчку 

```
corpus_ = sorted(corpus,key=corpus.get,reverse=True)
```

на

```
corpus_ = sorted(corpus,key=corpus.get,reverse=True)[:1000]
```


In [ ]:
import pandas as pd
import numpy as np
import json
import re
from string import punctuation
from collections import Counter
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from tqdm import tqdm
import random
random.seed(33)
torch.manual_seed(0)

In [ ]:
# Монтируем Гугл-диск
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Подготовка данных

In [ ]:
# Считываем данные из файла в две переменные: labels[] и reviews[]
labels = []
reviews = []
path='gdrive/My Drive/Data/ranking_train.jsonl'
def get_data_from_json(path: str) -> pd.DataFrame:
    with open(path, "r") as json_file:
        for str in json_file:
            data=json.loads(str)
            for comment in data["comments"]:
                reviews.append(data["text"]+comment["text"]+'\n\b')
                labels.append(comment["score"])
get_data_from_json(path)

In [ ]:
# Напечатаем первые 300 символов из файла с отзывами и 10 символов из файла с лейблами
print(reviews[:300])
print(labels[:10])
print(len(reviews))
print(len(labels[:]))

['How many summer Y Combinator fundees decided not to continue with their startup and go back to school? and what were the reasons?Going back to school is not identical with giving up. Some founders go back to school and keep working on the startup while there.  However, those do so much worse than the people who work on the startup full-time that going back to school seems, in practice, not too far removed from a death sentence for a startup.Off the top of my head, I\'d guess we\'ve had about 8 startups where the founders went back to school.  It doesn\'t only happen with summer batches.  Founders from winter batches do it too.Usually the reason is that the startup isn\'t doing very well. However, that judgement depends a lot on how determined the founders are.  One reason we now shy away from funding people still in school is that they often unconsciously want the startup to fail, because the idea of dropping out frightens them.A lot of startups look bad after 3 months.  Someone who\

In [ ]:
def preprocess(text):
    """"
    Функция чтобы разбить сплошной текст на отдельные отзывы, убрать пунктуацию
    и выделить все слова
    """
    text = "".join([s for s in text if s not in punctuation]) # убираем пунктуацию
    all_reviews = text.split("\n\b") # разделяем текст на ревью по знаку "\n\b"
    text = " ".join(all_reviews)
    all_words = text.split() # получаем массив слов
    
    return all_reviews, all_words

all_reviews, all_words = preprocess(reviews)
all_reviews=reviews
print('Общее число отзывов: ', len(all_reviews))

Общее число отзывов:  440535


In [ ]:
print('Первые 2 отзыва: ', all_reviews[:2])
print('Первые 5 слов: ', all_words[:5])


Первые 2 отзыва:  ['How many summer Y Combinator fundees decided not to continue with their startup and go back to school? and what were the reasons?Going back to school is not identical with giving up. Some founders go back to school and keep working on the startup while there.  However, those do so much worse than the people who work on the startup full-time that going back to school seems, in practice, not too far removed from a death sentence for a startup.Off the top of my head, I\'d guess we\'ve had about 8 startups where the founders went back to school.  It doesn\'t only happen with summer batches.  Founders from winter batches do it too.Usually the reason is that the startup isn\'t doing very well. However, that judgement depends a lot on how determined the founders are.  One reason we now shy away from funding people still in school is that they often unconsciously want the startup to fail, because the idea of dropping out frightens them.A lot of startups look bad after 3 mon

## Обработка данных

In [ ]:
corpus = Counter(all_words)
# Сортировка слов по встречаемости
corpus_ = sorted(corpus,key=corpus.get,reverse=True)[10:1000000]
print('Самые частые слова: ', corpus_[:10])
# кодируем каждое слово - присваиваем ему порядковый номер
vocab_to_int = {w:i+1 for i,w in enumerate(corpus_)}
print('Уникальных слов: ', len(vocab_to_int))

# Кодируем все отзывы: последовательность слов --> последовательность чисел
encoded_reviews = []
for sent in all_reviews:
  encoded_reviews.append([vocab_to_int[word] for word in sent.lower().split() 
                                  if word in vocab_to_int.keys()])
print('Пример закодированного коммента: ', encoded_reviews[0])

Самые частые слова:  ['the', 'to', 'a', 'of', 'and', 'is', 'that', 'I', 'in', 'for']
Уникальных слов:  1000000
Пример закодированного коммента:  [53, 103, 3024, 5574, 34359, 152021, 878, 20, 2, 858, 14, 43, 279, 5, 125, 156, 2, 21949, 5, 41, 99, 1, 156, 2, 513, 6, 20, 4389, 14, 724, 856, 46, 1326, 125, 156, 2, 513, 5, 243, 181, 13, 1, 279, 184, 776, 1583, 118, 38, 45, 70, 1149, 51, 1, 42, 63, 85, 13, 1, 279, 3422, 7, 104, 156, 2, 513, 14022, 9, 4598, 20, 151, 264, 2295, 31, 3, 2063, 2702, 10, 3, 1, 339, 4, 29, 6437, 10270, 480, 1707, 84, 33, 882, 616, 106, 1, 1326, 440, 156, 2, 3114, 12, 153, 73, 755, 14, 3024, 64982, 1326, 31, 7685, 23931, 38, 12, 1, 294, 6, 7, 1, 279, 230, 144, 74, 678, 1583, 7, 7874, 1415, 3, 105, 13, 53, 4699, 1, 1326, 1991, 50, 294, 56, 135, 8922, 344, 31, 1401, 42, 109, 9, 513, 6, 7, 24, 296, 27669, 83, 1, 279, 2, 6070, 67, 1, 186, 4, 3663, 52, 54619, 105, 4, 616, 180, 260, 157, 376, 2862, 139, 2369, 52, 4, 513, 5, 49, 2, 68, 12, 85, 22, 47, 3, 234, 9, 3, 21078, 

In [ ]:
encoded_labels = labels
print('Число отзывов и число лейблов: ', len(all_reviews), len(encoded_labels))

Число отзывов и число лейблов:  440535 440535


In [ ]:
# Убираем комменты длиной 0
encoded_labels=np.array( [label for idx, label in enumerate(encoded_labels) if len(encoded_reviews[idx]) > 0] )
encoded_reviews=[review for review in encoded_reviews if len(review) > 0]

print(len(encoded_labels), len(encoded_reviews))

440513 440513


Упакуем все последовательности, чтобы они имели одинаковую длину seq_len и могли быть организованы в тензоры по батчам

In [ ]:
def pad_text(encoded_reviews, seq_length):
    reviews = []
    for review in encoded_reviews:
        if len(review) >= seq_length:
            reviews.append(review[:seq_length])
        else:
            reviews.append([0]*(seq_length-len(review)) + review)
        
    return np.array(reviews)


padded_reviews = pad_text(encoded_reviews, seq_length = 200)

In [ ]:
dataset = TensorDataset(torch.from_numpy(padded_reviews), torch.from_numpy(encoded_labels))

#train_set = TensorDataset(torch.from_numpy(padded_reviews[:20000]), torch.from_numpy(encoded_labels[:20000]))
#val_set = TensorDataset(torch.from_numpy(padded_reviews[20000:]), torch.from_numpy(encoded_labels[20000:]))

In [ ]:
batch_size = 128
# 5000 на val конечно мало - надо добавить
train_set, val_set = torch.utils.data.random_split(dataset, [len(dataset)-40000, 40000])
print('Размер обучающего и валидационного датасета: ', len(train_set), len(val_set))
loaders = {'training': DataLoader(train_set, batch_size, pin_memory=True,num_workers=2, shuffle=True),
           'validation':DataLoader(val_set, batch_size, pin_memory=True,num_workers=2, shuffle=False)}

Размер обучающего и валидационного датасета:  400513 40000


In [ ]:
device = 'cuda' #if torch.cuda.is_available() else 'cpu'
print(device)

cuda


### Сборка модели для классификации текстов

In [ ]:
class SentimentRNN(nn.Module):
   
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers):
        """
        Инициализируем модель, обозначая слои и гиперпараметры
        """
        super(SentimentRNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, num_layers = num_layers)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, 1)
        self.sig = nn.Sigmoid()

    def forward(self, x, h):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, h)
        # print(lstm_out.shape)
        lstm_out = lstm_out[:, -1, :] # getting the last time step output
        
        # fully-connected layer
        out = self.fc(lstm_out)
        # sigmoid function
        out = self.sig(out)
        # return last sigmoid output
        return out

In [ ]:
vocab_size = len(vocab_to_int)+1
embedding_dim = 100
hidden_dim = 256
num_layers = 1
model = SentimentRNN(vocab_size, embedding_dim, hidden_dim, num_layers)
model.to(device)

optimizer = torch.optim.Adam(params = model.parameters()) # алгоритм оптимизации
lr = 0.002 # learning rate

### Обучаем модель

In [ ]:
class ValueMeter(object):
  """
  Вспомогательный класс, чтобы отслеживать loss и метрику
  """
  def __init__(self):
      self.sum = 0
      self.total = 0

  def add(self, value, n):
      self.sum += value*n
      self.total += n

  def value(self):
      return self.sum/self.total

def log(mode, epoch, loss_meter, accuracy_meter, best_perf=None):
  """
  Вспомогательная функция, чтобы 
  """
  print(
      f"[{mode}] Epoch: {epoch:0.2f}. "
      f"Loss: {loss_meter.value():.2f}. "
      f"Accuracy: {100*accuracy_meter.value():.2f}% ", end="\n")

  if best_perf:
      print(f"[best: {best_perf:0.2f}]%", end="")

In [ ]:
def accuracy(outputs, labels):
    preds = torch.round(outputs.squeeze())
    # print(preds, labels)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
def trainval(model, loaders, optimizer, epochs=5):
  """
  model: модель, которую собираемся обучать
  loaders: dict с dataloader'ами для обучения и валидации
  optimizer: оптимизатор
  epochs: число обучающих эпох (сколько раз пройдемся по всему датасету)
  """
  loss_meter = {'training': ValueMeter(), 'validation': ValueMeter()}
  accuracy_meter = {'training': ValueMeter(), 'validation': ValueMeter()}

  loss_track = {'training': [], 'validation': []}
  accuracy_track = {'training': [], 'validation': []}

  for epoch in range(epochs): # итерации по эпохам
      for mode in ['training', 'validation']: # обучение - валидация
          # считаем градиаент только при обучении:
          with torch.set_grad_enabled(mode == 'training'):
              # в зависимоти от фазы переводим модель в нужный ружим:
              model.train() if mode == 'training' else model.eval()
              for texts, labels in tqdm(loaders[mode]):
                  texts = texts.to(device) # отправляем тензор на GPU
                  labels = labels.to(device) 
                  bs = labels.shape[0]  # размер батча (отличается для последнего батча в лоадере)

                  zero_init = torch.zeros(num_layers,bs,hidden_dim).to(device)
                  h = tuple([zero_init, zero_init]) # задаем изначальный hidden state модели

                  preds = model(texts, h) # forward pass - прогоняем тензор через модель
                  loss = nn.BCELoss()(preds.squeeze(), labels.float())
                  loss_meter[mode].add(loss.item(), bs)

                  # если мы в фазе обучения
                  if mode == 'training':
                      optimizer.zero_grad() # обнуляем прошлый градиент
                      loss.backward() # делаем backward pass (считаем градиент)
                      optimizer.step() # обновляем веса
                  
                  acc = accuracy(preds, labels) # считаем метрику
                  # храним loss и accuracy для батча
                  accuracy_meter[mode].add(acc, bs)
                  
          # в конце фазы выводим значения loss и accuracy
          log(mode, epoch, loss_meter[mode], accuracy_meter[mode])

          # сохраняем результаты по всем эпохам
          loss_track[mode].append(loss_meter[mode].value())
          accuracy_track[mode].append(accuracy_meter[mode].value())
  return loss_track, accuracy_track

In [ ]:
loss_track, accuracy_track = trainval(model, loaders, optimizer, epochs=5)

### Смотрим результаты

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.plot(accuracy_track['training'], label='train')
plt.plot(accuracy_track['validation'], label='val')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.grid()
plt.legend()

In [ ]:
plt.plot(loss_track['training'], label='train')
plt.plot(loss_track['validation'], label='val')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.grid()
plt.legend()

In [ ]:
# Заморозка весов
for name, param in model.named_parameters():
    if name in ['fc.weight', 'fc.bias']:
        param.requires_grad = True
    else:
        param.requires_grad = False

## А дальше работа с тестовой выборкой ...

Вместо review нужно вставить тестовые данные

In [ ]:
# Загрузка тестовых данных
review_test = []
path='gdrive/My Drive/Data/ranking_test.jsonl'

def get_data_from_json(path: str) -> pd.DataFrame:
    with open(path, "r") as json_file:
        for str in json_file:
            data=json.loads(str)
            for comment in data["comments"]:
                reviewreview_test.append(data["text"]+comment["text"]+'\n\b')
get_data_from_json(path)
print(reviewreview_test[:300])
print(len(reviewreview_test))

In [ ]:
pred = []

def predict(model, review, seq_length = 200):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    _, words = preprocess(review.lower())
    encoded_words = [vocab_to_int[word] for word in words if word in vocab_to_int.keys()]
    padded_words = pad_text([encoded_words], seq_length)
    padded_words = torch.from_numpy(padded_words).to(device)
    bs=1        # размер батча был 1, так как на тесте гонялось по одному комментарию
    model.eval()
    zero_init = torch.zeros(num_layers,bs,hidden_dim).to(device)
    h = tuple([zero_init, zero_init])
    output = model(padded_words, h)
    pred.append(torch.trunc(5*output))
    # print(pred, '\n')

predict(model, review_test)  
# В итоге получим pred = [1, 4, 2, 0, 0, 3,... и т.д.] - все лейблы
# Надо как-то соединить их с комментами из файла ranking_test.jsonl
# Допустим сделать датафрейм и выгрузить его в файл .csv
# Или не надо